### Open a browser and set the cookies from a JSON file

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time,json, random, re, datetime
import pandas as pd
pd.options.mode.copy_on_write = True # recommended - https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

# ChromeDriver should match browser version. If outdated download from:
# https://googlechromelabs.github.io/chrome-for-testing/

def setCookiesFromJson():  
    try:
        DRIVER.get(BASE_URL) #RUN BROWSER
        currentUrlDomain = DRIVER.current_url
        currentUrlDomain = re.search(r'^https?://([^/]+)', currentUrlDomain)
        currentUrlDomain = currentUrlDomain.group(1)  
        currentUrlDomain = re.sub(r'^www\.', '', currentUrlDomain)
        currentUrlDomain = re.sub(r'^\.', '', currentUrlDomain)
        # print(currentUrlDomain)
        with open('cookies.json', 'r', newline='') as inputdata:
            cookies = json.load(inputdata)
            cookiesAdded = 0
            for cookie in cookies: #works only after driver.get
                if re.match(r".?"+currentUrlDomain, cookie['domain']): # can only add cookies for current domain
                    DRIVER.add_cookie(cookie)
                    cookiesAdded += 1
            if cookiesAdded > 0:
                DRIVER.refresh() # to load cookies
                return {'success':True, 'functionDone':True, 'message':'cookies for ' + currentUrlDomain + ' successfully set'}
            elif (cookiesAdded == 0):
                return {'success':False, 'functionDone':True, 'message':'no cookies for ' + currentUrlDomain + ' found in cookies.json'}
    except Exception as exception:
        return {'success':False, 'functionDone':True, 'message':str(exception)} # 'functionDone':True because it's not necessary

service = Service(executable_path="chromedriver.exe")
chrome_options = Options()

chrome_options.add_argument("--disable-search-engine-choice-screen")
chrome_options.add_argument("window-size=800,1000")
# chrome_options.add_experimental_option('excludeSwitches', ['enable-logging']) #disable error logging
DRIVER = webdriver.Chrome(service=service, options=chrome_options)

BASE_URL = "https://justjoin.it/job-offers/bialystok"
DRIVER.get(BASE_URL)
setCookiesFromJson()

{'success': True,
 'functionDone': True,
 'message': 'cookies for justjoin.it successfully set'}

### Fetch the URLs from all the pages

In [2]:
# def getLastOfferIndex():
#     try:
#         DRIVER.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll to the bottom
#         offersList = DRIVER.find_element(By.ID, 'up-offers-list')
#         offers = offersList.find_elements(By.XPATH, '//li[@data-index]') # amount depends on screen height 
#         lastIndex = offers[-1].get_attribute('data-index')
#         return lastIndex
#     except Exception as e:
#         print(e)
#         return

def offerNotFound():
    try:
        offerContent = DRIVER.find_element(By.CSS_SELECTOR, '.MuiBox-root.css-tnvghs')
        topContainer = offerContent.find_element(By.CSS_SELECTOR, 'div') # 1st div
        # NEED TO FIND 'OFFER NOT FOUND MSG AND CHECK WHICH DIVS DOES IT HAVE
        # topDiv = topContainer.find_element(By.XPATH, ".//*[contains(@class, 'css-10x887j')]")
        return False # if topDiv found, offer is there
    except:
        return True
# offerNotFound()

def anyOffersOnTheList():
    try:
        # offersList = DRIVER.find_element(By.ID, 'up-offers-list') # changed back to virtuoso ~15.01.2025 xD
        # offers = offersList.find_elements(By.XPATH, '//li[@data-index]') # changed back to virtuoso ~15.01.2025 xD

        offersList = DRIVER.find_element(By.XPATH, '//*[@data-test-id="virtuoso-item-list"]') # changed to up-offers-list ~25.12.2024
        offers = offersList.find_elements(By.XPATH, 'div[@data-index]') # virtuoso approach
        print(len(offers))
        if len(offers) > 0:
            return True
        else:
            return False
    except Exception as e:
        # print(e)
        return False

offersUrlsWithIndexesList = [{'index':-1, 'url':''}] # this element is deleted in fetchAllOffersUrls()

def fetchCurrentlyVisibleOffersUrls(): # just the ones currently rendered in browser
    try:
        offersList = DRIVER.find_element(By.XPATH, '//*[@data-test-id="virtuoso-item-list"]') # changed to up-offers-list ~25.12.2024
        offers = offersList.find_elements(By.XPATH, 'div[@data-index]') # amount depends on screen height 
        for offer in offers: # ever-loading div among them
            try:
                index = offer.get_attribute('data-index')
                href = offer.find_element(By.XPATH, ".//div/div/a").get_property("href")

                def foundAmongSavedIndexes():
                    for i in range (len(offersUrlsWithIndexesList[-30:])): # 30 last offers (or less if len < 30)
                        if index == offersUrlsWithIndexesList[-i - 1]['index']: # decrementing from the end
                            return True
                    return False # not found if reached this return
                
                if not foundAmongSavedIndexes():
                    offersUrlsWithIndexesList.append({'index':index, 'url':href})
                    
            except:
                pass #if url not found
        if len(offersUrlsWithIndexesList) >=1:
            print(int(offersUrlsWithIndexesList[-1]['index']) - int(offersUrlsWithIndexesList[0]['index']) + 1, len(offersUrlsWithIndexesList))
            print('first and last offersUrlsWithIndexesList: ', offersUrlsWithIndexesList[0]['index'], offersUrlsWithIndexesList[-1]['index'])
    except Exception as e:
        print(e)
        return

def fetchAllOffersUrls():
    noNewResultsCounter = 0
    lastSeenIndex = 0
    if not anyOffersOnTheList():
        return
    
    DRIVER.execute_script("window.scrollTo(0, 0);") # scroll to the top
    time.sleep(0.5)

    while True:
        # print(lastSeenIndex)
        fetchCurrentlyVisibleOffersUrls() #updates offersUrlsWithIndexesList

        if (lastSeenIndex == offersUrlsWithIndexesList[-1]['index']):
            noNewResultsCounter += 1
            DRIVER.execute_script("window.scrollBy(0, -2*innerHeight);") # for some reason scrolling up helps this fucking site to load the bottom
            time.sleep(1)
            DRIVER.execute_script("window.scrollBy(0, 3*innerHeight);") # scroll to the bottom
            print('noNewResults')
            # print(offersUrlsWithIndexesList[0]['index'], offersUrlsWithIndexesList[-1]['index'])
        else: # reset counter
            noNewResultsCounter = 0
        if noNewResultsCounter >= 5: # or int(lastSeenIndex) >= 10: # END IF NO NEW RESULTS FEW TIMES
            # fetchCurrentlyVisibleOffersUrls()
            del offersUrlsWithIndexesList[0] # this was a placeholder to avoid more ifs
            print('RETURN')
            #print(int(offersUrlsWithIndexesList[-1]['index']) - int(offersUrlsWithIndexesList[0]['index']) + 1, len(offersUrlsWithIndexesList))
            print('first and last offersUrlsWithIndexesList: ', offersUrlsWithIndexesList[0]['index'], offersUrlsWithIndexesList[-1]['index'])
            return
        
        lastSeenIndex = offersUrlsWithIndexesList[-1]['index']
        time.sleep(0.5)
        DRIVER.execute_script("window.scrollBy(0, innerHeight);")

fetchAllOffersUrls()
# fetchCurrentlyVisibleOffersUrls()
# print(offersUrlsWithIndexesList)

100
26 26
first and last offersUrlsWithIndexesList:  -1 24
26 26
first and last offersUrlsWithIndexesList:  -1 24
noNewResults
64 54
first and last offersUrlsWithIndexesList:  -1 62
64 54
first and last offersUrlsWithIndexesList:  -1 62
noNewResults
102 83
first and last offersUrlsWithIndexesList:  -1 100
109 90
first and last offersUrlsWithIndexesList:  -1 107
109 90
first and last offersUrlsWithIndexesList:  -1 107
noNewResults
109 90
first and last offersUrlsWithIndexesList:  -1 107
noNewResults
109 90
first and last offersUrlsWithIndexesList:  -1 107
noNewResults
109 90
first and last offersUrlsWithIndexesList:  -1 107
noNewResults
109 90
first and last offersUrlsWithIndexesList:  -1 107
noNewResults
RETURN
first and last offersUrlsWithIndexesList:  0 107


### Analyse offer functions

In [4]:
# try:
#     jobTitle = DRIVER.find_elements(By.CSS_SELECTOR, '.css-sy4ig6')
#     print(jobTitle)
#     jobTitle = jobTitle.text
# except:
#     jobTitle = None
# print(jobTitle)

# singleLocation = https://justjoin.it/job-offer/emagine-polska-technical-architect-warszawa-architecture

def getOfferDetails():
    # BASIC PARAMETERS WHICH SHOULD ALWAYS BE NOT EMPTY ON THE SITE
    try:
        offerContent = DRIVER.find_element(By.CSS_SELECTOR, '.MuiBox-root.css-tnvghs')
        topContainer = offerContent.find_element(By.CSS_SELECTOR, 'div')
        topDiv = topContainer.find_element(By.XPATH, ".//*[contains(@class, 'css-10x887j')]") # .// = as deep as necessary
    except Exception as exception:
        # print(exception)
        return # no point of continuing
    try:
        jobTitle = topDiv.find_element(By.CSS_SELECTOR, 'h1').text
        # print(jobTitle)
        employerAndLocationDiv = topDiv.find_element(By.CSS_SELECTOR, '.MuiBox-root.css-yd5zxy') 
        employer = employerAndLocationDiv.find_element(By.XPATH, './/h2').text # look for h2 as deep as necessary
        # print(employer) # name="multilocation_button"
    except:
        jobTitle, employer = '', ''

    try:
        location = employerAndLocationDiv.find_elements(By.CSS_SELECTOR, '.MuiBox-root.css-mswf74')[1].text # first one is employer
        location = re.sub(r'\+[0-9]+$', '', location) #remove '+x' where x is int
    except:
        location = ''
    #try clicking for more locations
    try:
        locationButton = employerAndLocationDiv.find_element("xpath", '//*[@name="multilocation_button"]')
        locationButton.click()
        locationsMenu = offerContent.find_element("xpath", '//ul[@role="menu"]')
        # locationsMenu = locationsMenu.find_elements(By.CSS_SELECTOR, 'li')
        location += '\n' + locationsMenu.text # TEXT EMPTY WHEN MINIMIZED!
    except Exception as exception:
        pass
    # print(location)

    #SALARY
    try:
        salaryAndContract = topContainer.find_element(By.CSS_SELECTOR, '.MuiBox-root.css-1km0bek').text
    except:
        salaryAndContract= ''

    salaryMinAndMax = [None, None] # Nones as these are INTs in DB
    if salaryAndContract != '':
        try: #to recalculate salary to [PLN/month net]
            grossToNetMultiplier = 0.7
            hoursPerMonthInFullTimeJob = 168
            lines = salaryAndContract.splitlines()[0] # There could be multiple salaries depending on contract type though. It will be in salaryAndContract anyway
            splitValues = re.split(r'-', lines) # split on dash for min and max

            for i in range(len(splitValues)):
                splitValues[i] = splitValues[i].replace(" ", "") # remove spaces
                splitValues[i] = re.sub(r",\d{1,2}", '', splitValues[i]) # removes , and /d{1 to 2 occurrences}  (needed when salary as 123,45)
                salaryMinAndMax[i] = re.search(r"\d+", splitValues[i]).group() # r = raw, \d+ = at least 1 digit, group() contains results
                
            if re.findall("brutto", lines[1]) or re.findall("gross", lines[1]): # gross -> net
                salaryMinAndMax = [(float(elmnt) * grossToNetMultiplier) for elmnt in salaryMinAndMax]
                # print(salaryMinAndMax)
            if re.findall("godz", lines[1]) or re.findall("hr.", lines[1]): # hr -> month
                salaryMinAndMax = [(float(elmnt) * hoursPerMonthInFullTimeJob) for elmnt in salaryMinAndMax] #possible input float/str

            salaryMinAndMax = [int(elmnt) for elmnt in salaryMinAndMax] # to ints
        except Exception as exception:
            pass    # salaryMinAndMax = [None, None]
    # print(salaryMinAndMax)

    # print(salaryAndContract)
    workModes = ''
    positionLevels = ''

    try:
        # MuiBox-root css-ktfb40
        fourRectanglesContainer = offerContent.find_elements(By.XPATH, "./div")[1] # only child divs, not grandchild or further - 1 level down
        # print(fourRectanglesContainer.text)
        fourRectangles = fourRectanglesContainer.find_elements(By.XPATH, "./div")

        for i in range(len(fourRectangles)):
            fourRectangles[i] = fourRectangles[i].find_elements(By.XPATH, "./div")[1] # second child div (not grandchild or further)
            fourRectangles[i] = fourRectangles[i].find_elements(By.XPATH, "./div")[1].text
            # print(fourRectangles[i])
        salaryAndContract += '\n' + fourRectangles[0] + ' | ' + fourRectangles[2]
        positionLevels = fourRectangles[1]
        workModes = fourRectangles[3]
    except Exception as exception:
        print(exception)
        pass
    # print(salaryAndContract)
    # print(workModes, positionLevels + '\n')

    #TECHSTACK
    techstackExpected, techstackOptional = '', ''
    try:
        techstackDiv = offerContent.find_elements(By.CSS_SELECTOR, '.MuiBox-root.css-qal8sw')[0]
        techstackDiv = techstackDiv.find_element(By.CSS_SELECTOR, 'div')
        technologies = techstackDiv.find_elements(By.XPATH, './/h4') # look for h4 in all children elements
        levels = techstackDiv.find_elements(By.XPATH, './/span') # look for h4 in all children elements
        for i in range(len(technologies)):
            techWithLvl = technologies[i].text + ' - ' + levels[i].text
            # print(techWithLvl)
            if levels[i].text == 'Nice To Have': # or levels[i].text == 'Junior'
                techstackOptional += '\n' + techWithLvl
            else: # -(nice to have)/junior/regular/advanced/master
                techstackExpected += '\n' + techWithLvl

        techstackOptional = re.sub(r"^\n", '', techstackOptional)
        techstackExpected = re.sub(r"^\n", '', techstackExpected)
    except Exception as exception:
        print(exception)
        pass # leave empty strs
    # print(techstackExpected + '\n\n' + techstackOptional)

    # ==================== DO THIS NOW ====================
    offerValidTo = '' # REMOVE FROM DB?
    fullDescription = '' # ADD TO DB
    responsibilities, requirements, optionalRequirements, theyOffer = '', '', '', ''

    optionalRequirementsKeywords = ['nice to', 'optional', 'ideal', 'preferr', 'asset', 'appreciat', 'atut', 'dodatk', 'mile widzi']
    requirementsKeywords = ['require', 'expect', 'skill', 'look', 'qualifications', 'experience', 'must', 'competen', 'wymaga', 'oczek', 'umiejętn', 'aplikuj, jeśli', 'oczekuj', 'potrzeb', 'szukamy', 'kompeten']
    responsibilitiesKeywords = ['responsib', 'task', 'role', 'project', 'obowiązk', 'zadani', 'projek']
    whatTheyOfferKeywords = ['offer', 'benefit' 'oferuj', 'oferow']

    allKeywordsDict = {'optionalRequirementsKeywords':optionalRequirementsKeywords, 'requirementsKeywords':requirementsKeywords, 'responsibilitiesKeywords':responsibilitiesKeywords}

    try:
        descriptionDiv = offerContent.find_elements(By.CSS_SELECTOR, '.MuiBox-root.css-qal8sw')[1]
        descriptionDiv = descriptionDiv.find_elements(By.XPATH, "./div")[1] # second child div
        fullDescription = descriptionDiv.text
        # print(r"{}".format(fullDescription))
        paragraphs = re.split(r"\n{2,}", fullDescription) # split on 2 or more newlines

        # USUALLY THE ABOVE SPLITTING IS ENOUGH, BUT IF NOT TRY FINDING PARAGRAPHS ANOTHER WAY
        if len(paragraphs) < 3: # 1 is possible minimum
            # print('LEN TOO SHORT: ' + str(len(paragraphs)))
            # paragraphs = re.split(r"(^|\n) +", fullDescription) # split on 1 or more spaces at the line beginning
            paragraphs = re.split(r"((^|\n) +|\n{2,})", fullDescription) # split on 1 or more spaces at the line beginning OR on 2 or more newlines
        # print(paragraphs)

        for paragraph in paragraphs:
            if not paragraph or re.search(r"^\s*$", paragraph): # \s matches Unicode whitespace characters. This includes [ \t\n\r\f\v] and more
                continue # don't try to analyze an empty sting, go with next loop iteration
            # look for keywords in the 1st line of text
            header = paragraph.splitlines()[0] # first line
            # print('=====================')
            # print(header)
            # print(paragraph)
            for keywordsCategory in allKeywordsDict.keys():
                # print(keywordsCategory)
                for keyword in allKeywordsDict[keywordsCategory]:
                    # if keyword found in header
                    # if re.search(rf'\b{re.escape(keyword)}\b', header, re.IGNORECASE): # \b = boundaries - matches whole words, regardless of punctuation or position in the string # escapes = escape regex reserved symbols
                    if re.search(rf'.*{re.escape(keyword)}.*', header, re.IGNORECASE): # .* = any symbol any number of times
                        # print('found ' + keyword)
                        if keywordsCategory =='optionalRequirementsKeywords': # check this first as it's more specific than requirements and contains similar keywords
                            optionalRequirements += paragraph
                        elif keywordsCategory =='requirementsKeywords':
                            requirements += paragraph
                        elif keywordsCategory =='responsibilitiesKeywords':
                            responsibilities += paragraph
                        elif keywordsCategory == 'whatTheyOfferKeywords':
                            whatTheyOfferKeywords += paragraph
    except Exception as exception:
        print(exception)
        pass

    # print('\n\n'+ responsibilities +'\n\n'+ requirements +'\n\n'+ optionalRequirements)
    datetimeNow = str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    return {'datetimeLast':datetimeNow, 'datetimeFirst':datetimeNow, 'url':DRIVER.current_url, 'title':jobTitle, 'salaryAndContract':salaryAndContract, 'salaryMin':salaryMinAndMax[0], 'salaryMax':salaryMinAndMax[1], 'employer':employer, 'workModes':workModes, 'positionLevels':positionLevels, 'location':location, 'techstackExpected':techstackExpected, 'techstackOptional':techstackOptional, 'responsibilities':responsibilities, 'requirements':requirements, 'optionalRequirements':optionalRequirements, 'fullDescription':fullDescription}
    

# testing below
# DRIVER.get('https://justjoin.it/job-offer/jit-team-senior-frontend-developer-bialystok-javascript')
# DRIVER.switch_to.window(DRIVER.window_handles[-1])
# getOfferDetails()

### Database management functions

In [67]:
# print(resultsDataFrame.employer)
# # resultsDataFrame.to_sql('offers', 'resultsDf.db') #alchemy needed
import sqlite3, re, datetime
import pandas as pd
from settings import DATABASE_TABLE_NAME, DATABASE_COLUMNS

class Database():
    def createTableIfNotExists(): #if not exists
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()

        # PREPARE STRING TO CREATE DB FROM
        databaseColumnsCreationString = ''
        for key, value in DATABASE_COLUMNS.items():
            databaseColumnsCreationString += f"{key} {value}, " # append k-v pair with comma and space
        databaseColumnsCreationString = re.sub(r',\s*$', '', databaseColumnsCreationString) # remove the comma and space(s) from the end

        # EXECUTE COMMAND
        cursor.execute("CREATE TABLE IF NOT EXISTS " + DATABASE_TABLE_NAME + "("+ databaseColumnsCreationString +");")
        connection.commit()
        cursor.close()
        connection.close()

    def selectAll():
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM" + DATABASE_TABLE_NAME +";")
        connection.commit()
        print(cursor.fetchall())
        cursor.close()
        connection.close()

    def executeQuery(query):
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        # print(cursor.fetchall())
        cursor.close()
        connection.close()
    
    def recordFound(url):
        urlPartToCompare = re.split("[?]s=", url)[0] #split on '?s=' because after that it's only session related stuff. If no pattern found url unchanged
        # print(urlPartToCompare)
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("SELECT datetimeFirst FROM " + DATABASE_TABLE_NAME + " WHERE url LIKE ('%" + urlPartToCompare + "%');")
        connection.commit()
        result = cursor.fetchall()
        cursor.close()
        connection.close()
        if len(result) >0:
            return True
        else:
            return False

    def insertRecord(dictionary):
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        # PREPARE COLUMN NAMES STR
        columnValuesString = ''
        for name in DATABASE_COLUMNS.keys():
            columnValuesString += f":{name}, "
        columnNamesString = re.sub(r',\s*$', '', columnNamesString) # remove the comma and space(s) from the end
        # EXECUTE INSERT
        cursor.execute("INSERT INTO " + DATABASE_TABLE_NAME + " VALUES ("+columnNamesString+")", dictionary)
        connection.commit()
        cursor.close()
        connection.close()

    def updateDatetimeLast(url):
        urlPartToCompare = re.split("[?]s=", url)[0] #split on '?s=' because after that it's only session related stuff. If no pattern found url unchanged
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("UPDATE " + DATABASE_TABLE_NAME + " SET datetimeLast = '" + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) + "'  WHERE url LIKE ('%" + urlPartToCompare + "%');")
        # cursor.execute("SELECT datetimeLast FROM " + DATABASE_TABLE_NAME + " WHERE url LIKE ('%" + urlPartToCompare + "%');")
        connection.commit()
        cursor.close()
        connection.close()
    
    def countAllRecords():
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("SELECT COUNT (*) FROM " + DATABASE_TABLE_NAME +";")
        connection.commit()
        resultTuple = cursor.fetchall()[0]
        (count,) = resultTuple #unpacking tuple
        cursor.close()
        connection.close()
        return str(count)

    def queryToDataframe(fullQuery):
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        # df = pd.read_sql("SELECT datetimeFirst, datetimeLast FROM " +DATABASE_TABLE_NAME+ ";", con=connection)
        df = pd.read_sql(fullQuery, con=connection)
        connection.commit()
        # print(cursor.fetchall())
        # print('\n'+str(len(cursor.fetchall())) + ' records found')
        cursor.close()
        connection.close()
        return df

Database.createTableIfNotExists()
Database.countAllRecords()

'59'

### Scraping to database

In [6]:
columnsAll = ['datetimeFirst', 'datetimeLast', 'url', 'title', 'salaryAndContract', 'salaryMin', 'salaryMax', 'employer', 'workModes', 'positionLevels', 'offerValidTo', 'location', 'techstackExpected', 'techstackOptional', 'responsibilities', 'requirements', 'optionalRequirements'] # move out of global scope later

import numpy as np

def scrapToDatabase():
    timeDeltas = []
    inserts = 0
    updates = 0
    print(Database.countAllRecords() + ' records before run')
    # for i in range (0,2):
    for i in range (len(offersUrlsWithIndexesList)):
        DRIVER.get(offersUrlsWithIndexesList[i]['url'])
        if not offerNotFound():
            resultsList = getOfferDetails()
            print(resultsList)
            outputDictionary = {}
            for column, offerDetail in zip(columnsAll, resultsList):
                outputDictionary[column] = offerDetail #combine 2 lists into 1 dictionary
            before = time.time()
            if Database.recordFound(DRIVER.current_url):
                Database.updateDatetimeLast(DRIVER.current_url)
                # print(driver.current_url)
                updates += 1
            else:
                Database.insertRecord(outputDictionary) # insert into databas
                inserts += 1
                # print('insert')
            timeDeltas.append(time.time() - before)
            #ending here and starting in an above for/zip loop it takes ~(1/100)s - good enough
            print (str(i+1) + '/' + str(len(offersUrlsWithIndexesList)) + ' done')
        else:
            print('OFFER NOT FOUND: ' +  DRIVER.current_url)
        time.sleep(random.uniform(0.35,0.85)) #Humanize requests frequency
    print(np.mean(timeDeltas))
    print(str(inserts) + ' inserts | ' + str(updates) + ' updates')

scrapToDatabase()

1085 records before run
['2025-01-17 12:09:52', '2025-01-17 12:09:52', 'https://justjoin.it/job-offer/antal-sp-z-o-o--senior-mdm-developer-bialystok-java', 'Senior MDM Developer', '\nFull-time | B2B', None, None, 'Antal Sp. z o.o.', 'Remote', 'Senior', '', 'Białystok, ', 'Java - Master\nKubernetes - Master\nAzure - Master\nApache Kafka - Advanced\n.Net - Advanced', '', 'Role Overview\nWe are seeking a Senior MDM Developer to join our DevOps team. The ideal candidate will have extensive experience in the following areas:\nJava backend development with strong expertise in the Spring Boot framework.\nAzure development and Kubernetes for container orchestration.\nDatabase planning and implementation, e.g., PostgreSQL.\nModern integration technologies such as Kafka and REST APIs. Bonus points for experience with GraphQL.\nDevelopment and implementation of MDM solutions.\nFamiliarity with front-end technologies and languages (JavaScript, TypeScript).', '', '']
1/89 done
['2025-01-17 12:09:55

In [6]:
from settings import DATABASE_COLUMNS, DATABASE_DEFAULT_TEXT, DATABASE_DEFAULT_INT, DATABASE_TABLE_NAME
import re

# PREPARE STRING TO CREATE DB FROM
databaseColumnsCreationString = ''
for key, value in DATABASE_COLUMNS.items():
    databaseColumnsCreationString += f"""{key} {value['dataType']} DEFAULT {value['default']}, """ 
databaseColumnsCreationString = re.sub(r',\s*$', '', databaseColumnsCreationString) # remove the comma and space(s) from the end
print(databaseColumnsCreationString)

datetimeLast TEXT DEFAULT "", datetimeFirst TEXT DEFAULT "", url TEXT DEFAULT "", title TEXT DEFAULT "", salaryAndContract TEXT DEFAULT "", salaryMin INT DEFAULT NULL, salaryMax INT DEFAULT NULL, employer TEXT DEFAULT "", workModes TEXT DEFAULT "", positionLevels TEXT DEFAULT "", location TEXT DEFAULT "", techstackExpected TEXT DEFAULT "", techstackOptional TEXT DEFAULT "", responsibilities TEXT DEFAULT "", requirements TEXT DEFAULT "", optionalRequirements TEXT DEFAULT "", fullDescription TEXT DEFAULT ""


In [8]:
# from flask import Flask, render_template, request, send_file
# from bokeh.plotting import figure
# from bokeh.resources import CDN
# from bokeh.models.widgets import DataTable, TableColumn
# from bokeh.models import ColumnDataSource, WheelZoomTool, HTMLTemplateFormatter, HoverTool, TapTool, Range1d, LinearAxis
# from bokeh.embed import json_item
# from bokeh.io import curdoc #for dark theme
# import io #for a csv buffer

# def makeBokehPlot(dataframe): #Only offers with specified salary?
#     # len(dataframe) >=1 at this point 
#     # dataframe already ordered by (salaryMin+SalaryMax)/2 ASC

#     pd.options.mode.copy_on_write = True #recommended - https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
#     pd.set_option('future.no_silent_downcasting', True)

#     nonNanRowsDf = dataframe[dataframe['salaryMin'].notna()]
#     nanRowsDf = dataframe[dataframe['salaryMin'].isna()]

#     # SPECIFY UNSPECIFIED BARS HEIGHT
#     if len(nonNanRowsDf) > 0: #otherwise division by 0 possible
#         lookUpToValues = 2 #how many values to count average
#         avgOfNLowestMinSalaries = nonNanRowsDf['salaryMin'].head(lookUpToValues).tolist() #select up to 2 values
#         avgOfNLowestMinSalaries = sum(avgOfNLowestMinSalaries) / len(avgOfNLowestMinSalaries) #avg
#         avgOfNLowestMaxSalaries = nonNanRowsDf['salaryMax'].head(lookUpToValues).tolist() #select up to 2 values
#         avgOfNLowestMaxSalaries = sum(avgOfNLowestMaxSalaries) / len(avgOfNLowestMaxSalaries) #avg
#         nanRowsDf['salaryMin'] = nanRowsDf['salaryMin'].fillna(avgOfNLowestMinSalaries) #replace nulls with values
#         nanRowsDf['salaryMax'] = nanRowsDf['salaryMax'].fillna(avgOfNLowestMaxSalaries)
#     else: #if only unspecified salaries foud
#         avgOfNLowestMinSalaries = 4200 #some value to plot
#         avgOfNLowestMaxSalaries = 4200
#         nanRowsDf['salaryMin'] = nanRowsDf['salaryMin'].fillna(avgOfNLowestMinSalaries) #replace nulls with values
#         nanRowsDf['salaryMax'] = nanRowsDf['salaryMax'].fillna(avgOfNLowestMaxSalaries)
        
#     dataSalaryUnspecified = {
#         'x': nanRowsDf.index.tolist(),
#         'title': nanRowsDf['title'].values.tolist(),
#         'activeFor': [(dtstr.days) for dtstr in (pd.to_datetime(nanRowsDf["datetimeLast"])-pd.to_datetime(nanRowsDf["datetimeFirst"])).tolist()], #.days shows only days
#         'salaryAvg': [((avgOfNLowestMinSalaries+avgOfNLowestMaxSalaries)/2) for i in range (len(nanRowsDf))]
#     }
#     dataSalarySpecified = {
#         'x': nonNanRowsDf.index.tolist(),
#         'title': nonNanRowsDf['title'].values.tolist(),
#         'activeFor': [(dtstr.days) for dtstr in (pd.to_datetime(nonNanRowsDf["datetimeLast"])-pd.to_datetime(nonNanRowsDf["datetimeFirst"])).tolist()], #.days shows only days
#         'salaryMin': nonNanRowsDf['salaryMin'].values.tolist(),
#         'salaryMax': nonNanRowsDf['salaryMax'].values.tolist(),
#         'salaryAvg': [(a + b) / 2 for a, b in zip(nonNanRowsDf['salaryMin'].values.tolist(), nonNanRowsDf['salaryMax'].values.tolist())],
#     }

#     #Calculate ranges - SAFE MAX by declaring default values used if empty list
#     maxActiveFor = int(max(max(dataSalaryUnspecified['activeFor'], default=0) , max(dataSalarySpecified['activeFor'], default=0))) +1 #
#     maxSalary = max(max(dataSalaryUnspecified['salaryAvg'], default=0) , max(dataSalarySpecified['salaryMax'], default=0)) * 1.05

#     sourceSalaryUnspecified = ColumnDataSource(dataSalaryUnspecified) #2 data sources
#     sourceSalarySpecified = ColumnDataSource(dataSalarySpecified) #2 data sources
#     plot = figure(title="", x_axis_label='Offer index', y_axis_label='Salary', height = 400, sizing_mode='stretch_width')
#     plot.y_range = Range1d(start=0 - 1, end=maxSalary) # * 1.2 to fit the bars
#     plot.x_range = Range1d(start=0 - 1, end=int(len(dataframe))) #too much empty space by default
#     plot.extra_y_ranges = {"y2": Range1d(start=0, end=maxActiveFor)} #add 1 day
#     #COLORS
#     salaryUnspecifiedColor = 'rgb(60,60,160)'
#     salarySpecifiedColor = 'rgb(80,80,220)'
#     # daysActiveColor = 'rgb(30,150,30)'
#     daysActiveColor = 'rgb(60,100,40)'
#     # SALARY UNSPECIFIED BARS
#     plot.vbar('x', top = 'salaryAvg', width = 0.70, source = sourceSalaryUnspecified, color=salaryUnspecifiedColor, alpha = 1) # MAIN BAR
#     plot.vbar('x', top = 'activeFor', y_range_name="y2", source = sourceSalaryUnspecified, color=daysActiveColor, alpha = 0.15, width=0.90) # Active for
#     # plot.segment(x0='x', y0='salaryMin', x1='x', y1='salaryMax', source=sourceSalaryUnspecified, line_width=2, color='black', alpha = 0.5) #Error bar
#     # SALARY SPECIFIED BARS
#     plot.vbar('x', top = 'salaryAvg', width = 0.70, source = sourceSalarySpecified, color=salarySpecifiedColor, alpha = 1) # MAIN BAR
#     plot.vbar('x', top = 'activeFor', y_range_name="y2", source = sourceSalarySpecified, color=daysActiveColor, alpha = 0.15, width=0.90) # Active for
#     plot.segment(x0='x', y0='salaryMin', x1='x', y1='salaryMax', source=sourceSalarySpecified, line_width=1.5, color='black', alpha=0.75) #Error bar
    
#     plot.add_layout(LinearAxis(y_range_name="y2", axis_label="Days adtive"), 'right') # Add the second y-axis to the right
    
#     # Configure minor gridlines
#     plot.xgrid.minor_grid_line_color = 'rgb(80,80,80)'
#     plot.ygrid.minor_grid_line_color = 'rgb(80,80,80)'
#     plot.xgrid.minor_grid_line_alpha = 0.5 # Opacity
#     plot.ygrid.minor_grid_line_alpha = 0.5

#     taptool = TapTool() #highlight on tap
#     wheel_zoom = WheelZoomTool()
#     plot.toolbar.active_scroll = wheel_zoom
#     hoverSalaryUnpecified = HoverTool(tooltips=[("Offer index:", "@x"), ("Job title:", "@title"), ("Salary:", "Unspecified"), ("Active for:", "@activeFor days")])
#     hoverSalaryUnpecified.renderers = [plot.renderers[0]]# hover tool only on the salary bars
#     hoverSalarySpecified = HoverTool(tooltips=[("Offer index:", "@x"), ("Job title:", "@title"), ("Min/Avg/Max:", "@salaryMin{0.}/@salaryAvg{0.}/@salaryMax{0.}"), ("Active for:", "@activeFor days")]) #{0} = no decimals
#     hoverSalarySpecified.renderers = [plot.renderers[2]]# hover tool only on the salary bars
#     plot.add_tools(hoverSalarySpecified, hoverSalaryUnpecified, taptool) #wheel_zoom removed for now
#     #DARK THEME
#     curdoc().theme = 'dark_minimal'
#     curdoc().add_root(plot) #to apply the theme
#     return plot

# def makeBokehTable(dataframe):
#     source = ColumnDataSource(dataframe)
#     columns = []
#     for column in dataframe.columns:
#         if column == 'url': # to make a hyperlink
#             columns.append(TableColumn(field=column, title=column, formatter=HTMLTemplateFormatter(template="""<a href="<%= value %>" target="_blank"><%= value %></a>""")))
#         else:
#             columns.append(TableColumn(field=column, title=column))     
#     table = DataTable(source=source, columns=columns, height = 800, editable=True, sizing_mode="stretch_width")
#     return table

# app = Flask(__name__)

# @app.route('/downloadCsv')
# def downloadCsv():
#     # Save the DataFrame to a CSV file
#     csvName = "jobScrappingResults " + str(datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")) + ".csv"
#     # Save the DataFrame to a CSV in memory
#     buffer = io.BytesIO() #buffer for a csv file to avoid saving csv on a disk
#     dataframeTable.to_csv(buffer, sep=',', encoding='utf-8-sig', index=True, header=True)
#     buffer.seek(0)  # Reset buffer position to the beginning
#     # Send the CSV file as a downloadable response
#     return send_file(buffer, as_attachment=True, download_name=csvName, mimetype='text/csv')

# @app.route('/', methods=['GET', 'POST'])
# def form():
#     if request.method == 'GET':
#         # return render_template("form.html", columnsAll=columnsAll)
#         return render_template("app.html", columnsAll=columnsAll, resources=CDN.render())
    
#     elif request.method == 'POST':
#         def makeFormOutputDictionary():
#             formDictFromJson = request.get_json() #get form values from a request
#             outputDict = {}
#             for column in columnsAll:
#                 rowDictionary = {'show': False, 'necessary': None, 'forbidden': None, 'above': None, 'below': None}
#                 #show column
#                 if formDictFromJson.get(column+'Show', False): #if not found assign False. Found only if form field not empty
#                     rowDictionary['show'] = True
#                 #necessary phrase
#                 if formDictFromJson.get(column+'Necessary', False):
#                     phraseNecessary = formDictFromJson.get(column+'Necessary')
#                     # phraseNecessary = phraseNecessary.split(", ") #delete
#                     rowDictionary['necessary'] = phraseNecessary
#                 #forbidden phrase
#                 if formDictFromJson.get(column+'Forbidden', False):
#                     phraseForbidden = formDictFromJson.get(column+'Forbidden')
#                     # phraseForbidden = phraseForbidden.split(", ") #delete
#                     rowDictionary['forbidden'] = phraseForbidden
#                 #above
#                 if formDictFromJson.get(column+'Above', False):
#                     rowDictionary['above'] = formDictFromJson.get(column+'Above')
#                     # print('found ' + column+'Above') #
#                 #below
#                 if formDictFromJson.get(column+'Below', False):
#                     rowDictionary['below'] = formDictFromJson.get(column+'Below')
#                     # print('found ' + column+'Below') #
#                 outputDict[column] = rowDictionary #append row with column name as a key
#             # print(outputDict)
#             return outputDict
        
#         def queryBuilder(formDictionary):
            
#             def handleBracketsAndLogicalOperators(input, param, like):
#                 if like:
#                     likePart = ' LIKE '
#                 elif not like:
#                     likePart = ' NOT LIKE '
#                 splittedResults = re.split(r" OR | AND ", input) #split on logic operator
#                 phrases = []
#                 for res in splittedResults:
#                     res = re.sub(r'\(|\)', '', res) #remove brackets
#                     res = re.sub(r'^ +| +$', '', res) #remove spaces at both ends
#                     phrases.append(res)
#                 for phrase in phrases: #make placeholders one by one
#                     input = re.sub(phrase, '<<<>>>', input, count=1) #count=1 to only replace the first match. This is needed because phrases content can overlap
#                 for phrase in phrases: #fill placeholders one by one
#                     input = re.sub('<<<>>>', param + likePart + "('%" +phrase+"%')", input, count=1) #only first match
#                 return input

#             querySelectPart = "SELECT "
#             queryMainPart = "\nWHERE 1=1" #removing this later
#             for columnName in formDictionary.keys():
#                 currentColumnDictionary = formDictionary[columnName].items()
#                 for key, value in currentColumnDictionary:
#                     # SELECT STATEMENT APPENDING
#                     if key == 'show' and value:
#                         querySelectPart += columnName + ', '
#                     #ABOVE & BELOW 
#                     if key == 'above' and value:
#                         queryMainPart += "\nAND "+columnName+" > '"+value+"'"
#                     if key == 'below' and value:
#                         queryMainPart += "\nAND "+columnName+" < '"+value+"'"
#                     #NECESSARY PHRASE
#                     if key == 'necessary' and value: # if list not empty
#                         queryMainPart += "\nAND "+ handleBracketsAndLogicalOperators(value, columnName, like=True)
#                     #FORBIDDEN PHRASE
#                     if key == "forbidden" and value:
#                         queryMainPart += "\nAND "+ handleBracketsAndLogicalOperators(value, columnName, like=False)
#             queryMainPart += '\nORDER BY (salaryMin+SalaryMax)/2 ASC, (JULIANDAY(datetimeLast) - JULIANDAY(datetimeFirst)) * 24 * 60 DESC;' #order by

#             querySelectPart = re.sub(r", $", '', querySelectPart) #remove ", " from the end
#             querySelectPart += " FROM "+tableName # 1=1 to append only ANDs
#             queryMainPart = re.sub(r" 1=1\nAND", '', queryMainPart) #remove "1=1\nAND" if at least 1 filter specified
#             queryMainPart = re.sub(r"\nWHERE 1=1", '', queryMainPart)# or remove WHERE 1=1 if no filters specified. If specified shouldn't match this regexp
#             query = querySelectPart + queryMainPart
#             queryPlot = "SELECT datetimeFirst, datetimeLast, title, salaryMin, salaryMax FROM "+ tableName + queryMainPart #2nd query - always select datetimes and salaries for plotting, order by time active and avg salary
#             # print('\n'+query+'\n'+queryPlot)
#             return query, queryPlot
        
#         global dataframeTable #to make it accessible to download at all times
#         dataframeTable, dataframePlot = queryBuilder(makeFormOutputDictionary())
#         dataframeTable = database.queryToDataframe(dataframeTable)
#         dataframePlot = database.queryToDataframe(dataframePlot)

#         if len(dataframePlot) > 0 and len(dataframeTable) > 0: #tho their lengths should be equal
#             plot = makeBokehPlot(dataframePlot)
#             table = makeBokehTable(dataframeTable)
#             return json.dumps([json_item(plot), json_item(table), int(len(dataframeTable))])

#         return json.dumps(['noResultsFound']) #when no results return a str

# if __name__ == "__main__":
#     app.run(debug=True, use_reloader=False)#JUPYTER